In [1]:
# !pip install undetected-chromedriver

In [2]:
import numpy as np
import pandas  as pd
import time
import traceback

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
def create_session_like_driver():
    options = uc.ChromeOptions()

    # user-agent + language
    options.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    )
    options.add_argument("--lang=vi,en-US;q=0.9,en;q=0.8")

    # Một số option chống phát hiện cơ bản
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    # options.add_argument('--headless')   # bật nếu muốn chạy không mở cửa sổ

    # # Tắt load ảnh (cách đúng: dùng prefs)
    # prefs = {"profile.managed_default_content_settings.images": 2}
    # options.add_experimental_option("prefs", prefs)


    driver = uc.Chrome(options=options)
    return driver

# 1.Điền thông tin tk mk fb clon

In [4]:
email = "njnjakhai@gmail.com"
password = "Njnjakhai01@"

In [5]:
def login_fb(driver):
    driver.get("https://www.facebook.com/")
    wait = WebDriverWait(driver, 20)

    # dien thong tin tai khoan
    email_box = wait.until(EC.presence_of_element_located((By.ID, "email")))
    email_box.send_keys(email)
    time.sleep(1)

    # dien thong tin mk
    pass_box = wait.until(EC.presence_of_element_located((By.ID, "pass")))
    pass_box.send_keys(password)
    pass_box.send_keys(Keys.ENTER)

    wait.until(lambda d: "login" not in d.current_url.lower())

    time.sleep(60)

# Nhập link bài báo

In [17]:
list_link = [
    "https://www.facebook.com/groups/408571091380061/posts/1190688756501620/",
    "https://www.facebook.com/groups/408571091380061/posts/1187024023534760/",
    "https://www.facebook.com/groups/408571091380061/posts/1191317279772101/",
    "https://www.facebook.com/groups/408571091380061/posts/1190783459825483/",
    "https://www.facebook.com/groups/408571091380061/posts/836761268561039/",

]


## Sau khi truy cập fb phải tắt thủ công thông báo và thông báo lưu mật khẩu vì sẽ làm hỏng tiến trình, có 20s sleep để tự tay tắt

In [18]:
# Basic setup
driver = create_session_like_driver()

login_fb(driver=driver)

for i, link in enumerate(list_link):
    print(f"Xử lý link {i + 1}")
    driver.get(link)
    wait = WebDriverWait(driver, 20)

    time.sleep(2)

    # Ána nút hiển thị tất cả comment trong bài viết
    # 1 số bài đăng quá ngắn nên không ấn được nút, cho và try-except để bỏ qua nếu không ấn được
    try:
        comment_button = wait.until(EC.element_to_be_clickable((
            By.XPATH,
            "/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[2]/div/div/div/div/div/div/div/div[2]/div[2]/div/div/div/div/div/div/div/div/div/div/div/div/div[13]/div/div/div[4]/div/div/div[2]/div[1]/div/div/span"
        )))
        comment_button.click()
        time.sleep(5)

        show_all_comment = wait.until(EC.presence_of_all_elements_located((
            By.XPATH, 
            "/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div/div/div[1]/div[1]/div/div/div/div/div/div/div[1]/div/div[3]/div[1]/div/div[1]/span"
        )))
        show_all_comment[0].click()
        time.sleep(5)
    except Exception as e:
        print("Lỗi không thể ấn nút")

    # scroll_and_scraper_comment
    batch_size = 10
    previous_comment_count = 0
    no_change_count = 0
    comments = []

    for i in range(10):
        for batch in range(batch_size):
            # Scroll
            driver.execute_script("""
                document.querySelectorAll('*').forEach(el => {
                    if (el.scrollHeight > el.clientHeight) {
                        el.scrollTop += 1200;
                    }
                });
            """)
            time.sleep(3)
            
            # Đếm comments hiện tại
            current_comments = driver.find_elements(By.CSS_SELECTOR, 'span[dir="auto"]')
            current_count = len(current_comments)
            
            print(f"Batch {i+1} - lần {batch+1}: {current_count} comments")
            
            # Kiểm tra nếu đếm comment trong batch không tăng, chuyển sang batch tiếp
            if current_count == previous_comment_count:
                no_change_count += 1
                if no_change_count >= 3:  # 3 lần liên tiếp không tăng
                    print("✅ Không có comment mới trong batch")
                    break
            else:
                no_change_count = 0  # Reset counter
            
            previous_comment_count = current_count
            
            # Scraper comment và lưu ngay trong batch
            current_container_comments = driver.find_elements(
                By.CSS_SELECTOR,
                'span[dir="auto"]'
            )

            for container in current_container_comments:
                div_elements = container.find_elements(
                    By.CSS_SELECTOR,
                    'div[dir="auto"]'
                )
                
                full_text = []
                for div in div_elements:
                    text = div.text.strip()
                    if text:
                        full_text.append(text)  

                if full_text:
                    combine_text = '\n'.join(full_text)
                    if len(combine_text) > 5:
                        comments.append({
                            'text': combine_text,
                            'length': len(combine_text)
                        })

    time.sleep(20)

    print("-----------------------------------------------------")

    # Lọc ra comment unique trong list comment trùng nhau
    unique_comments = []
    seen_comments = set()

    for item in comments:
        if item['text'] not in seen_comments:
            unique_comments.append(item)
            seen_comments.add(item['text'])

    print(f"Tổng số comment unique: {len(unique_comments)}")

    df = pd.DataFrame(unique_comments)
    df.to_csv("comment_data.csv", mode='a', header=False, index=False)

    print("Đã lưu comment vào CSV")

driver.quit()

Xử lý link 1
Batch 1 - lần 1: 169 comments
Batch 1 - lần 2: 169 comments
Batch 1 - lần 3: 169 comments
Batch 1 - lần 4: 169 comments
✅ Không có comment mới trong batch
Batch 2 - lần 1: 169 comments
✅ Không có comment mới trong batch
Batch 3 - lần 1: 169 comments
✅ Không có comment mới trong batch
Batch 4 - lần 1: 169 comments
✅ Không có comment mới trong batch
Batch 5 - lần 1: 169 comments
✅ Không có comment mới trong batch
Batch 6 - lần 1: 169 comments
✅ Không có comment mới trong batch
Batch 7 - lần 1: 169 comments
✅ Không có comment mới trong batch
Batch 8 - lần 1: 169 comments
✅ Không có comment mới trong batch
Batch 9 - lần 1: 169 comments
✅ Không có comment mới trong batch
Batch 10 - lần 1: 169 comments
✅ Không có comment mới trong batch
-----------------------------------------------------
Tổng số comment unique: 6
Đã lưu comment vào CSV
Xử lý link 2
Batch 1 - lần 1: 189 comments
Batch 1 - lần 2: 189 comments
Batch 1 - lần 3: 189 comments
Batch 1 - lần 4: 189 comments
✅ Không có

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=49279): Read timed out. (read timeout=120)

In [ ]:
import pandas as pd

In [ ]:
test = pd.read_csv("comment_data.csv")
test

,text,length
0,CHÚC MỪNG SINH VIÊN PTIT TỎA SÁNG VỚI BA GIẢI ...,2865
1,Chúc mừng zai mẹ nhé,20
2,Xin chúc mừng các bạn trẻ,25
3,1 bữa lẩu ở iec nhé\r\nNguyễn Trang,32
4,Nguyễn Trang\r\nchờ quà hcm lâu quá,32
...,...,...
5972,"xúc đi b, như mình nè, giờ bạn trọ của mình vớ...",94
5973,NA Quan\r\nah xin em,17
5974,Thèm thế three some đc k b,26
5975,Pháp luật nào cấm nứ iu lam định ní lào cấm la...,87


In [ ]:
test = test.drop(['length'], axis=1)
test

,text
0,CHÚC MỪNG SINH VIÊN PTIT TỎA SÁNG VỚI BA GIẢI ...
1,Chúc mừng zai mẹ nhé
2,Xin chúc mừng các bạn trẻ
3,1 bữa lẩu ở iec nhé\r\nNguyễn Trang
4,Nguyễn Trang\r\nchờ quà hcm lâu quá
...,...
5972,"xúc đi b, như mình nè, giờ bạn trọ của mình vớ..."
5973,NA Quan\r\nah xin em
5974,Thèm thế three some đc k b
5975,Pháp luật nào cấm nứ iu lam định ní lào cấm la...


In [ ]:
def remove_newlines_and_add_label(df):
    # Xóa tất cả dấu xuống dòng (cả \r và \n) trong từng sample text
    df['text'] = df['text'].astype(str).str.replace(r'[\r\n]+', ' ', regex=True)
    # Loại bỏ duplicate dựa trên cột 'text'
    df = df.drop_duplicates(subset=['text']).reset_index(drop=True)
    # Tạo thêm cột label với giá trị 0 cho toàn bộ dữ liệu
    df['label'] = 0
    return df


# Sử dụng hàm:
test = remove_newlines_and_add_label(test)

In [ ]:
test

,text,label
0,CHÚC MỪNG SINH VIÊN PTIT TỎA SÁNG VỚI BA GIẢI ...,0
1,Chúc mừng zai mẹ nhé,0
2,Xin chúc mừng các bạn trẻ,0
3,1 bữa lẩu ở iec nhé Nguyễn Trang,0
4,Nguyễn Trang chờ quà hcm lâu quá,0
...,...,...
5721,"xúc đi b, như mình nè, giờ bạn trọ của mình vớ...",0
5722,NA Quan ah xin em,0
5723,Thèm thế three some đc k b,0
5724,Pháp luật nào cấm nứ iu lam định ní lào cấm la...,0


In [ ]:
test.to_csv("comment.csv", encoding="utf-8", index=False)

NameError: name 'test' is not defined

In [ ]:
data_p = pd.read_csv("comment.csv")[2900:]
data_p

,text,label
2900,ảo,0
2901,=)))) cmn,1
2902,vcl,1
2903,=)))))))) Hả,1
2904,Vaicadai,1
...,...,...
3773,Nghỉ bão khum,0
3774,Thông báo ngày mai vẫn học bth nhé các em :)))))),0
3775,HỌC PHÍ,0
3776,Ngày nghỉ sẵn trên qldt rồi mong cho cb đi học,0


In [ ]:
out = data_p[data_p['label'] == 0].dropna(axis=0)

In [ ]:
out

,text,label
2900,ảo,0
2908,THÔNG BÁO VỀ VÒNG LOẠI ICPC PTIT 2025 - CƠ SỞ ...,0
2910,Ae nào thấy tên không vừa ý thì cứ tự nhiên lê...,0
2911,2 anh đi thi xong kéo e vào team lấy danh hiệu...,0
2912,Đặt cái tên tưởng thế là hay,0
...,...,...
3773,Nghỉ bão khum,0
3774,Thông báo ngày mai vẫn học bth nhé các em :)))))),0
3775,HỌC PHÍ,0
3776,Ngày nghỉ sẵn trên qldt rồi mong cho cb đi học,0


In [ ]:
out.to_csv("out_label_n.csv", encoding="utf-8", index=False)